In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
#     config = 'configs.config_WomenInEngineering_local'
    config = 'configs.config_ILLAE_local'
#     config = 'configs.config_Digits_local'    

In [2]:
### initialization ###

import importlib
print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_ILLAE_local


In [3]:
### creating the dynamic features folder ###

try:
    imp.os.makedirs('{}data/features/dynamic_features/'.format(c.directory['save']))
except OSError as e:
    if e.errno != imp.errno.EEXIST:
        raise Failed('Counld\'nt create tmp folder!')

In [4]:
### reading cv ids for cv folds ###

outer_ids = imp.pd.read_csv('{}{}'.format(c.directory['save'], 'cv_ids/outer_0.csv'))
outer_ids.rename(columns={'id': c.id_name}, inplace=True)

inner_ids = list()
for i in range(c.inner):
  inner_ids.append(imp.pd.read_csv('{}cv_ids/inner_0_{}.csv'.format(c.directory['save'], i)))

imp.pprint('Number of cv = {}'.format(len(inner_ids)))

'Number of cv = 5'


In [5]:
### for-loop for creating dyanmic features ###

def for_loop_dynamic_features(experiment, df_inputs, dynamics):

    cv=experiment['cv']
    training = imp.copy.deepcopy(inner_ids)
    del training[cv]
    training_ids=imp.pd.DataFrame()
    for item in training:
        training_ids=imp.pd.concat([training_ids, item], ignore_index=True, axis=0)
    training_ids.rename(columns={'id':c.id_name}, inplace=True)

    validation_ids = imp.copy.deepcopy(inner_ids[cv])
    validation_ids.rename(columns={'id':c.id_name}, inplace=True)


#     for feature in c.select_features['dynamic']:
#         if experiment['type'] == c.features['dynamic'][feature]['type']:
    feature=experiment['name']
    dynamic=dynamics[c.features['dynamic'][feature]['type']]
    df_input=df_inputs[feature]
    df_training=imp.pd.merge(df_input, training_ids, on=c.id_name, how='right')
    df_validation=imp.pd.merge(df_input, validation_ids, on=c.id_name, how='right')        

    dynamic.construct_features(c, experiment, feature,
                               df_training, df_validation,
                               training_ids, validation_ids)

In [6]:
### getting the parameter grid for the experiments ###

dfs=dict()
dfs['experiments']=imp.pd.read_csv('{}experiments/{}/1_parameters_dynamic_features.csv'.format(c.directory['save'],
                                                                              util.experiment_name(c)))
experiments_ids= list(range(c.index_start, len(dfs['experiments']), c.index_step))
dfs['experiments']=dfs['experiments'].iloc[experiments_ids]

imp.pprint('number of experiments = {}'.format(len(dfs['experiments'])))

util.display_df(dfs['experiments'])

'number of experiments = 5'


,cv,lda_hashtags_LDA_num_topics,lda_hashtags_LDA_topic_cutoff_threshold,lda_hashtags_no_above,lda_hashtags_no_below,name,type
0,0,20,0.1,0.6,2,lda_hashtags,lda
1,1,20,0.1,0.6,2,lda_hashtags,lda
2,2,20,0.1,0.6,2,lda_hashtags,lda
3,3,20,0.1,0.6,2,lda_hashtags,lda
4,4,20,0.1,0.6,2,lda_hashtags,lda


In [7]:
### loading dynamic feature modules ###:

dynamics=dict()
df_inputs=dict()
for feature in c.select_features['dynamic']:
    dynamics[c.features['dynamic'][feature]['type']]=importlib.import_module('dynamic_features.{}'.format(c.features['dynamic'][feature]['type']))
    importlib.reload(dynamics[c.features['dynamic'][feature]['type']])
    df_inputs[feature]=imp.pd.read_csv('{}data/features/{}_feature_{}({}).csv'.format(c.directory['save'],
                                                                                         c.project_name,
                                                                                         c.features['dynamic'][feature]['type'],
                                                                                         feature))
    df_inputs[feature]=dynamics[c.features['dynamic'][feature]['type']].prep_input(df_inputs[feature], c)

parallel: 15 partitions with 15 cores for para_parsing_df


In [8]:
### debugging purposes-1 (Experimental Setup) ###

# params=dict(
#             df_inputs=df_inputs, 
#             dynamics=dynamics
#            )

# print(params.keys())
# experiment=[exp for exp in dfs['experiments'].to_dict('records')][3]
# params_names='_'.join(str(experiment[experiment['name']+'_'+param_name]) for param_name in 
#                       c.features['dynamic'][experiment['name']]['params'].keys())    
# print('{}_{}({})_{}_cv({})'.format(c.project_name 
#                                , experiment['type']
#                                , experiment['name']
#                                , params_names
#                                , experiment['cv']
#                               ))
# '_'.join(list(map(str, list(experiment.values()))))
# print(experiment)

# for param_name in c.features['dynamic'][experiment['name']]['params'].keys():
#     key=experiment['name']+'_'+param_name
#     print(experiment[key])
# params_names='_'.join(str(experiment[experiment['name']+'_'+param_name]) for param_name in c.features['dynamic'][experiment['name']]['params'].keys())    
# params_names

In [9]:
### debugging purposes-2 (Feature Construction) ###

# cv=experiment['cv']
# training = imp.copy.deepcopy(inner_ids)
# del training[cv]
# training_ids=imp.pd.DataFrame()
# for item in training:
#     training_ids=imp.pd.concat([training_ids, item], ignore_index=True, axis=0)
# training_ids.rename(columns={'id':c.id_name}, inplace=True)

# validation_ids = imp.copy.deepcopy(inner_ids[cv])
# validation_ids.rename(columns={'id':c.id_name}, inplace=True)


# for feature in c.select_features['dynamic']:
#     if experiment['type'] == c.features['dynamic'][feature]['type']:
#         dynamic=dynamics[c.features['dynamic'][feature]['type']]
#         df_input=df_inputs[feature]
#         df_training=imp.pd.merge(df_input, training_ids, on=c.id_name, how='right')
#         df_validation=imp.pd.merge(df_input, validation_ids, on=c.id_name, how='right')        

#         dynamic.construct_features(c, experiment, feature,
#                                    df_training, df_validation,
#                                    training_ids, validation_ids)
#         df_dynamic=dynamic.retrieve_features(c, experiment, feature)
#         util.display(df_dynamic.head())


In [10]:
### debugging purposes-3 ###

# for_loop_dynamic_features(experiment, **params)

# OR 

# list(map(imp.functools.partial(for_loop_dynamic_features, **params), [experiment for experiment in dfs['experiments'].to_dict('records')]))

In [11]:
### running the experiments in parallel ###

result_cvs=imp.Parallel(n_jobs=c.num_cores
                    , prefer="processes"
                    , verbose=3
                   ) (imp.delayed(for_loop_dynamic_features)(experiment, 
                                                               df_inputs,
                                                               dynamics
                                                              ) 
                      for experiment in dfs['experiments'].to_dict('records'))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   2 out of   5 | elapsed:   28.2s remaining:   42.3s
[Parallel(n_jobs=15)]: Done   5 out of   5 | elapsed:   28.5s finished


In [12]:
### for-loop for all data - dyanmic features ###

def for_loop_dynamic_features_all(experiment, df_inputs, dynamics):

    training = imp.copy.deepcopy(inner_ids)
    training_ids=imp.pd.DataFrame()
    for item in training:
        training_ids=imp.pd.concat([training_ids, item], ignore_index=True, axis=0)
    training_ids.rename(columns={'id':c.id_name}, inplace=True)
    testing_ids = imp.copy.deepcopy(outer_ids)
        
#     for feature in c.select_features['dynamic']:
    feature=experiment['name']
    dynamic=dynamics[c.features['dynamic'][feature]['type']]
    print(c.features['dynamic'][feature]['type'])
    df_input=df_inputs[feature]
    df_training=imp.pd.merge(df_input, training_ids, on=c.id_name, how='right')
    df_testing=imp.pd.merge(df_input, testing_ids, on=c.id_name, how='right')        

    experiment_temp=imp.copy.deepcopy(experiment)
    experiment_temp['name']=feature
    experiment_temp['type']=c.features['dynamic'][feature]['type']
    experiment_temp['cv']='all'


    df_dynamic=dynamic.construct_features(c, experiment_temp, feature,
                                          df_training, df_testing,
                                          training_ids, testing_ids)
#     df_dynamic=dynamic.retrieve_features(c, experiment_temp, feature)
#     util.display(df_dynamic.head())

In [13]:
### running all in parallel ###

imp.Parallel(n_jobs=c.num_cores
                    , prefer="processes"
                    , verbose=3
                   ) (imp.delayed(for_loop_dynamic_features_all)(sub_exp[1].iloc[0,:], 
                                                                 df_inputs,
                                                                 dynamics
                                                              ) 
                      for sub_exp in dfs['experiments'].groupby('name'))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   1 out of   1 | elapsed:   17.3s finished


[None]

In [14]:
# for sub_exp in dfs['experiments'].groupby('name'):
# #     print(util.display_df(sub_exp[1]))
#     for_loop_dynamic_features_all(sub_exp[1].iloc[0,:],
#                                   df_inputs,
#                                   dynamics)